# MLP Project – Predicting Purchase Value from Session Data

##### Importing basic libraries

In [33]:
import pandas as pd
import numpy as np
import warnings

In [34]:
# Importing the training and testing dataset into df and dft
df = pd.read_csv('/kaggle/input/e-commerce-dataset/train_data.csv')
dft = pd.read_csv('/kaggle/input/e-commerce-dataset/test_data.csv')

# Setting option to display all the columns
pd.set_option('display.max_columns', None)

# To avoid warnings
warnings.filterwarnings("ignore")
df

,trafficSource.isTrueDirect,purchaseValue,browser,device.screenResolution,trafficSource.adContent,trafficSource.keyword,screenSize,geoCluster,trafficSource.adwordsClickInfo.slot,device.mobileDeviceBranding,device.mobileInputSelector,userId,trafficSource.campaign,device.mobileDeviceMarketingName,geoNetwork.networkDomain,gclIdPresent,device.operatingSystemVersion,sessionNumber,device.flashVersion,geoNetwork.region,trafficSource,totals.visits,geoNetwork.networkLocation,sessionId,os,geoNetwork.subContinent,trafficSource.medium,trafficSource.adwordsClickInfo.isVideoAd,browserMajor,locationCountry,device.browserSize,trafficSource.adwordsClickInfo.adNetworkType,socialEngagementType,geoNetwork.city,trafficSource.adwordsClickInfo.page,geoNetwork.metro,pageViews,locationZone,device.mobileDeviceModel,trafficSource.referralPath,totals.bounces,date,device.language,deviceType,userChannel,device.browserVersion,totalHits,device.screenColors,sessionStart,geoNetwork.continent,device.isMobile,new_visits
0,NaN,0.0,Edge,not available in demo dataset,NaN,NaN,medium,Region_2,NaN,not available in demo dataset,not available in demo dataset,61421,(not set),not available in demo dataset,domain1,0,not available in demo dataset,1,not available in demo dataset,Washington,youtube.com,1,not available in demo dataset,1500100799,Windows,Northern America,referral,NaN,not available in demo dataset,United States,not available in demo dataset,NaN,Not Socially Engaged,Redmond,NaN,Seattle-Tacoma WA,1.0,8,not available in demo dataset,/intl/hr/yt/about/,1.0,20170714,not available in demo dataset,desktop,Social,not available in demo dataset,1,not available in demo dataset,1500100799,Americas,False,1.0
1,True,0.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,not available in demo dataset,72287,(not set),not available in demo dataset,domain3,0,not available in demo dataset,1,not available in demo dataset,California,(direct),1,not available in demo dataset,1495262065,Macintosh,Northern America,(none),NaN,not available in demo dataset,United States,not available in demo dataset,NaN,Not Socially Engaged,Mountain View,NaN,San Francisco-Oakland-San Jose CA,1.0,8,not available in demo dataset,NaN,1.0,20170519,not available in demo dataset,desktop,Direct,not available in demo dataset,1,not available in demo dataset,1495262065,Americas,False,1.0
2,True,0.0,Chrome,not available in demo dataset,NaN,(not provided),medium,Region_2,NaN,not available in demo dataset,not available in demo dataset,25180,(not set),not available in demo dataset,domain1,0,not available in demo dataset,2,not available in demo dataset,Lombardy,google,1,not available in demo dataset,1508510328,Windows,Southern Europe,organic,NaN,not available in demo dataset,Italy,not available in demo dataset,NaN,Not Socially Engaged,Milan,NaN,(not set),6.0,8,not available in demo dataset,NaN,NaN,20171020,not available in demo dataset,desktop,Organic Search,not available in demo dataset,6,not available in demo dataset,1508510328,Europe,False,NaN
3,NaN,0.0,Internet Explorer,not available in demo dataset,NaN,NaN,medium,Region_4,NaN,not available in demo dataset,not available in demo dataset,41295,(not set),not available in demo dataset,domain3,0,not available in demo dataset,1,not available in demo dataset,not available in demo dataset,youtube.com,1,not available in demo dataset,1483431838,Windows,Eastern Asia,referral,NaN,not available in demo dataset,Japan,not available in demo dataset,NaN,Not Socially Engaged,not available in demo dataset,NaN,not available in demo dataset,1.0,8,not available in demo dataset,/yt/about/ja/,1.0,20170103,not available in demo dataset,desktop,Social,not available in demo dataset,1,not available in demo dataset,1483431838,Asia,False,1.0
4,True,88950000.0,Chrome,not available in demo dataset,NaN,NaN,medium,Region_3,NaN,not available in demo dataset,not available in demo dataset,113697,(not set),not available in demo dataset,domain1,0,not available 

## EDA

In [ ]:
null_cols = [
    'totals.visits', 'browserMajor', 'device.browserSize', 'device.screenResolution',
    'screenSize', 'device.language', 'device.screenColors', 'device.mobileDeviceModel',
    'device.mobileDeviceBranding', 'device.mobileInputSelector', 'device.mobileDeviceMarketingName',
    'device.operatingSystemVersion', 'device.flashVersion', 'geoNetwork.networkLocation',
    'locationZone', 'socialEngagementType', 'device.browserVersion'
]
print(f"{'Feature':35s}   {'Value':32s} {' Normalized Value'}")
for col in null_cols:
    vc = df[col].value_counts(normalize=True)
    if len(vc) == 1:
        val = vc.index[0]
        prop = vc.iloc[0]
        print(f"{col:35s} -> {str(val):32s} ({prop:.2f})")


Most of these features have only one unique value across the dataset, as shown in the output. These provide no useful variance for model training and can be safely dropped.

In [ ]:
for col in null_cols:
    df = df.drop(col, axis=1)
    dft=dft.drop(col, axis=1)

Dropping the above-mentioned columns from both training and test sets to clean the dataset and reduce noise.

In [ ]:
df=df.drop('userId',axis=1)
dft=dft.drop('userId',axis=1)
df=df.drop('sessionId',axis=1)
dft=dft.drop('sessionId',axis=1)

Removed userId and sessionId since they are identifiers and don't contribute any meaningful signal to the prediction task.

# Identifying and Handling Missing Values 

In [ ]:
null_values=df.isnull().sum()
null_values[null_values>0]

Removing features with more number of null values (>50%)

In [ ]:
null_cols=[
    'trafficSource.adContent',
    'trafficSource.adwordsClickInfo.slot',
    'trafficSource.adwordsClickInfo.isVideoAd',
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.page'
]
for col in null_cols:
    df = df.drop(col, axis=1)
    dft=dft.drop(col, axis=1)

In [ ]:
cols=['trafficSource.campaign',
       'trafficSource.keyword',
       'geoNetwork.metro']
print(f"{'Feature_Name':30s}{'Top Value':30s}","Normalized value")
for col in cols:
    print(f"{col:30s}{df[col].value_counts(normalize=True).index[0]:30s}",df[col].value_counts(normalize=True).iloc[0])

Since these columns are dominated by a single unknown value, they are not informative. We will remove them.

In [ ]:
for col in cols:
    df = df.drop(col, axis=1)
    dft=dft.drop(col, axis=1)

In [ ]:
df.isnull().sum()

These will be handled after train_test_split

**Feature Engineering** for the 'sessionStart' feature

In [ ]:
df['sessionStart']

In [ ]:
df['sessionStart_dt'] = pd.to_datetime(df['sessionStart'], unit='s')
df['sessionStart_dt']

Converting the session start timestamp to datetime format.

### Feature Engineering

In [ ]:
df['hour'] = df['sessionStart_dt'].dt.hour
df['dayofweek'] = df['sessionStart_dt'].dt.dayofweek
df['month'] = df['sessionStart_dt'].dt.month
df.drop(columns=['sessionStart', 'sessionStart_dt'], inplace=True)

#For test df
dft['sessionStart_dt'] = pd.to_datetime(dft['sessionStart'], unit='s')
dft['hour'] = dft['sessionStart_dt'].dt.hour
dft['dayofweek'] = dft['sessionStart_dt'].dt.dayofweek  # 0=Monday
dft['month'] = dft['sessionStart_dt'].dt.month
dft.drop(columns=['sessionStart', 'sessionStart_dt'], inplace=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select relevant columns
cols_to_check = ['purchaseValue', 'pageViews', 'totalHits']
corr_matrix = df[cols_to_check].corr()

# Display correlation coefficients
print("Correlation matrix:\n", corr_matrix)

# Plot heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title("Correlation Heatmap: purchaseValue vs Key Features")
plt.show()


Correlation Analysis with Numerical columns and Target variable

Target Variable Analysis

In [ ]:
df['purchaseValue'].value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
df['purchaseValue'].hist()

In [ ]:
import seaborn as sns
non_zero_values = df[df['purchaseValue'] >0]['purchaseValue']
plt.figure(figsize=(10, 6))
sns.kdeplot(non_zero_values, fill=True, color='skyblue', linewidth=2)
plt.title('Density Plot of Purchase Values (Excluding Zeros)')
plt.xlabel('Purchase Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

We can see that most of the values are closer to 0 or even equal to 0, but there are a wide variety of values

In [ ]:
log_values = np.log1p(df[df['purchaseValue'] > 0]['purchaseValue'])

plt.figure(figsize=(10, 6))
sns.kdeplot(log_values, fill=True, color='green')
plt.title('Density Plot of Log-Transformed Purchase Values')
plt.xlabel('Log(Purchase Value + 1)')
plt.grid(True)
plt.show()


Log Trannsformed Density Plot gives a clear understand of how this plot is actually distributed

### Insight:

The target purchaseValue is highly imbalanced with very large values.

~79% of the values are 0.

The remaining are very high numbers, making this both sparse and skewed.

## Preprocessing

Train Test Splitting

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Split data
X = df.drop('purchaseValue', axis=1)
y = df['purchaseValue']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

# Separating High and low Cardinality columns
cat_cols = X_train.select_dtypes(include='object').columns.tolist()
low_card = [col for col in cat_cols if X_train[col].nunique() <= 20]
high_card = [col for col in cat_cols if X_train[col].nunique() > 20]
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

if 'purchaseValue' in num_cols:
    num_cols.remove('purchaseValue')

#Filling the null values
fill_values = {}
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        fill_values[col] = X_train[col].mode()[0]
    else:
        fill_values[col] = X_train[col].mean()

X_train = X_train.fillna(fill_values)
X_test = X_test.fillna(fill_values)

# Frequency encode high-card cols
def frequency_encode(train, test, cols):
    for col in cols:
        freq = train[col].value_counts(normalize=True)
        train[col] = train[col].map(freq)
        test[col] = test[col].map(freq).fillna(0)
    return train, test

X_train, X_test = frequency_encode(X_train, X_test, high_card)

# Column Transformer
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False), low_card)
], remainder='passthrough')


Implementing the Pipeline

In [ ]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb

models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42),
    "XGBoost": xgb.XGBRegressor(random_state=42)
}

from sklearn.metrics import r2_score, mean_squared_error

results=[]
for name, model in models.items():
    pipe = Pipeline([
        ('preprocessing', preprocessor),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    results.append({
        'Model': name,
        'R2 Score': round(r2, 4),
        'MSE': round(mse, 2)
    })

# Convert to DataFrame
import pandas as pd
results_df = pd.DataFrame(results)

# Sort by best R2 Score
results_df = results_df.sort_values(by='R2 Score', ascending=False)

# Show table
print(results_df.to_markdown(index=False)) 

When we try a direct Regressor model, we are not getting proper R2 Scores,

### Dual model approach

First we do Classification to predict if purchase happened or not and then do Regression to predict the actual purchase value.

Classification – to predict whether a purchase occurs (0 or not)

Regression – to predict the actual purchase amount for non-zero sessions

### Preprocessing

Preprocessing in such a way to feed to a classifier model

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

def full_preprocessing(X_train, X_test, unique_threshold=20):
    X_train = X_train.copy()
    X_test = X_test.copy()

    # Handle Missing Values
    for col in X_train.columns:
        if X_train[col].dtype == 'object' or X_train[col].dtype.name == 'category':
            mode_val = X_train[col].mode()[0]
            X_train[col] = X_train[col].fillna(mode_val)
            X_test[col] = X_test[col].fillna(mode_val)
        else:
            mean_val = X_train[col].mean()
            X_train[col] = X_train[col].fillna(mean_val)
            X_test[col] = X_test[col].fillna(mean_val)

    # Identify Categorical Columns
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    low_card_cols = [col for col in categorical_cols if X_train[col].nunique() <= unique_threshold]
    high_card_cols = [col for col in categorical_cols if X_train[col].nunique() > unique_threshold]

    # Frequency Encoding
    def frequency_encode(df_train, df_test, columns):
        for col in columns:
            freq = df_train[col].value_counts() / len(df_train)
            df_train[col] = df_train[col].map(freq)
            df_test[col] = df_test[col].map(freq).fillna(0)
        return df_train, df_test

    X_train, X_test = frequency_encode(X_train, X_test, high_card_cols)

    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
    numeric_cols += high_card_cols  # include freq encoded

    # Define Column Transformer
    preprocessor = ColumnTransformer(transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), low_card_cols)
    ], remainder='passthrough')  # no scaling

    return X_train, X_test, preprocessor


Splitting and applying preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('purchaseValue', axis=1)
y = df['purchaseValue']

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=60)

X_train_proc, X_test_proc, preprocessor = full_preprocessing(X_train, X_test)
X_train_transformed = preprocessor.fit_transform(X_train_proc)
X_test_transformed = preprocessor.transform(X_test_proc)

#Labels for Classifier
y_train_class = (y_train > 0).astype(int)
y_test_class = (y_test > 0).astype(int)


### Training Classifier

HistGradientBoosting Classifier is the best classifier among all the classifiers I tried

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import f1_score

clf=HistGradientBoostingClassifier()
clf.fit(X_train_transformed, y_train_class)
threshold = 0.75
probs = clf.predict_proba(X_test_transformed)[:, 1]
stage1_preds = (probs > threshold).astype(int)
f1_score(y_test_class,stage1_preds)

Performing Hyper Parameter tuning for HGB Classifier

Using compute_class_weight library to handle the imbalance in the target var

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

#Weights for the imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_class), y=y_train_class)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_iter': [100, 200],
    'max_depth': [5,7],
    'l2_regularization': [1.0]
}

clf_hpt = HistGradientBoostingClassifier(
    random_state=42,
    class_weight=class_weight_dict
)

grid = GridSearchCV(clf_hpt, param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)
grid.fit(X_train_transformed, y_train_class)

print("Best params:", grid.best_params_)
print("Best F1 score:", grid.best_score_)


Now we get the best score for the classifier, we train the model with those parameters

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_class), y=y_train_class)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
clf = HistGradientBoostingClassifier(
    random_state=42,
    learning_rate=0.1,
    max_iter=200,
    max_depth=7,
    l2_regularization=1.0,
    class_weight=class_weight_dict
)
clf.fit(X_train_transformed, y_train_class)

threshold = 0.75
probs = clf.predict_proba(X_test_transformed)[:, 1]
stage1_preds = (probs > threshold).astype(int)

Coming to Regression of those predicted values

We do Random Forest Regressor - which overall was the best regressor (in this dual model pipeline) when I tried with other models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Higher weight for positive purchases
weights = np.where(y_train > 0, 20, 1)

regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_transformed, y_train,sample_weight=weights)

# Predict only for predicted positive sessions
regression_preds = regressor.predict(X_test_transformed[stage1_preds == 1])

# Merge predictions
final_preds = np.zeros(len(X_test_transformed))
final_preds[stage1_preds == 1] = regression_preds

# Evaluate
print("Final R2 Score:", r2_score(y_test, final_preds))

Performing HyperParameter Tuning for Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Filter only positive purchase sessions for training
mask_train = y_train > 0
X_train_pos = X_train_transformed[mask_train]
y_train_pos = y_train[mask_train]
weights_pos = np.where(y_train_pos > 0, 20, 1)  # same logic

# Grid Search params
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15],
    'min_samples_split': [2, 5],
}

reg = RandomForestRegressor(random_state=42)

grid = GridSearchCV(
    estimator=reg,
    param_grid=param_grid,
    cv=2,
    scoring='r2',
    verbose=1,
    n_jobs=-1
)

grid.fit(X_train_pos, y_train_pos, sample_weight=weights_pos)

best_rf = grid.best_estimator_

# Predict only for predicted positive sessions in test
regression_preds = best_rf.predict(X_test_transformed[stage1_preds == 1])

# Merge predictions
final_preds = np.zeros(len(X_test_transformed))
final_preds[stage1_preds == 1] = regression_preds

# Evaluate
from sklearn.metrics import r2_score
print("Final R2 Score on Full Test Set:", r2_score(y_test, final_preds))


We can notice there is poor performance when we do hyper parameter tuning for random forest, so we just pass the actual model with all default hyper parameters (as before) for better performance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Higher weight for positive purchases
weights = np.where(y_train > 0, 20, 1)

regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_transformed, y_train,sample_weight=weights)

# Predict only for predicted positive sessions
regression_preds = regressor.predict(X_test_transformed[stage1_preds == 1])

# Merge predictions
final_preds = np.zeros(len(X_test_transformed))
final_preds[stage1_preds == 1] = regression_preds

# Evaluate
print("Final R2 Score:", r2_score(y_test, final_preds))

The Final R2 Score with all default hyper parameters in random forest model

In [ ]:
from sklearn.metrics import r2_score

# Train R² (regression only on y_train > 0)
train_mask = y_train > 0
reg_train_preds = regressor.predict(X_train_transformed[train_mask])
train_r2 = r2_score(y_train[train_mask], reg_train_preds)
print("Train R²:", train_r2)

# Test R²
test_mask = stage1_preds == 1
reg_test_preds = regressor.predict(X_test_transformed[test_mask])
test_r2 = r2_score(y_test[test_mask], reg_test_preds)
print("Test R²:", test_r2)


This is the highest R2 for both train adn test I got, which explains least overfitting of the model

Other different models gave better training score but very low testing score, so that lead to overfitting of the model.

In [ ]:
X_dft = dft.copy()

_, X_dft_proc, _ = full_preprocessing(X_train.copy(), X_dft.copy(), unique_threshold=20)

X_dft_transformed = preprocessor.transform(X_dft_proc)

probs_dft = clf.predict_proba(X_dft_transformed)[:, 1]
stage1_preds = (probs_dft > threshold).astype(int)

regression_preds = regressor.predict(X_dft_transformed[stage1_preds == 1])

final_preds = np.zeros(len(X_dft_transformed))
final_preds[stage1_preds == 1] = regression_preds

submission = pd.DataFrame({
    'id': range(len(final_preds)),
    'purchaseValue': final_preds
})

submission.to_csv('submission.csv', index=False)
print("✅ submission.csv created successfully.")


This is the final submission file for the test dataset given for the competition.

Preprocessing dft (test df) and applying these models, and predicting the values for final submission.csv file